# Recommendation Engines: Implementing Surprise
- Surprise is a scikit library that stands for **Simple Python RecommendatIon System Engine**
- Has built-in similarity metrics, baseline methods, content-based systems, matrix factorization systems

In this notebook, we'll first walk through setting up a super basic recommendation system, using the popular MovieLens 100K Dataset. Then, we'll look into more detail how Surprise works.

## Fitting and Predicting with Surprise

### 1. Install surprise if you haven't, and import the usual libraries.

In [3]:
# import libraries
import numpy as np
import pandas as pd

from surprise import Dataset, Reader
from surprise import SVD
from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split

### 2. Load in the dataset

Surprise has the dataset built in. You might need to download the dataset so follow the instructions in the code output! Unfortunately, the Surprise data format doesn't let us inspect the data, but here is the documentation: https://grouplens.org/datasets/movielens/100k/


In [6]:
data = Dataset.load_builtin('ml-100k')

# train-test split
train, test = train_test_split(data, test_size=.2)

In [7]:
train

### 3. Run the default Singular Value Decomposition Model!

In [8]:
svd = SVD()
svd.fit(train)
predictions = svd.test(test)

In [9]:
accuracy.rmse(predictions)

RMSE: 0.9403


0.9403164596877142

### 4. Make a prediction!

In [10]:
uid = str(196)  # raw user id (as in the ratings file). They are **strings**!
iid = str(302)

# get a prediction for specific users and items.
pred = svd.predict(uid, iid, r_ui=4, verbose=True)

user: 196        item: 302        r_ui = 4.00   est = 4.25   {'was_impossible': False}


## Applying Surprise

### 1. How does Surprise take in your data?
https://surprise.readthedocs.io/en/stable/getting_started.html#use-a-custom-dataset

The dataset we'll use is a subset of the Yelp Open Dataset that's already been joined and cleaned.
https://www.yelp.com/dataset

In [11]:
yelp = pd.read_csv('yelp_reviews.csv').drop(['Unnamed: 0'], axis = 1)

In [12]:
yelp.head()

,user_id,business_id,stars
0,brd33PD_6nqK_VVnO3NWAg,--1UhMGODdWsrMastO9DZw,4.0
1,NqpKiaRsGfuU2voV5dPRCQ,--1UhMGODdWsrMastO9DZw,1.0
2,dhzlnpisqA7V1zfiO12AZA,--1UhMGODdWsrMastO9DZw,2.0
3,A4bpHuvzaQt9-XAg8e9Msw,--1UhMGODdWsrMastO9DZw,3.0
4,GL81ktDIteXA2VVH6gIakg,--1UhMGODdWsrMastO9DZw,5.0


### 2. Inspecting the dataset:

Here's where you'd do a **comprehensive** EDA!

In [13]:
print('Number of Users: ', len(yelp['user_id'].unique()))
print('Number of Businesses: ', len(yelp['business_id'].unique()))

Number of Users:  79773
Number of Businesses:  2518


1. What's the distribution of ratings? i.e. How many 1-star, 2-star, 3-star reviews?
2. How many reviews does a restaurant have?
3. How many reviews does a user make?

In [14]:
yelp['stars'].value_counts()

5.0    42685
4.0    23143
1.0    14315
3.0    11522
2.0     8335
Name: stars, dtype: int64

In [15]:
yelp['business_id'].value_counts()

-ed0Yc9on37RoIoG2ZgxBA    1694
--9e1ONYQuAa-CB_Rrw7Tw    1661
-6tvduBzjLI1ISfs3F_qTg    1194
-U7tvCtaraTQ9b0zBhpBMA    1180
-FLnsWAa4AGEW4NgE8Fqew    1128
                          ... 
-cOlxAJesBk0bYUb1dbn_A       3
-ChcBjI6WgpwOujPM-JkYA       3
-9MlHwqLZP7i1ed5NPFxyw       3
-888fFkw3R5YxjPJFdYeGg       3
-dfeuVFpFWB97xV8hW_EZg       3
Name: business_id, Length: 2518, dtype: int64

In [16]:
yelp

,user_id,business_id,stars
0,brd33PD_6nqK_VVnO3NWAg,--1UhMGODdWsrMastO9DZw,4.0
1,NqpKiaRsGfuU2voV5dPRCQ,--1UhMGODdWsrMastO9DZw,1.0
2,dhzlnpisqA7V1zfiO12AZA,--1UhMGODdWsrMastO9DZw,2.0
3,A4bpHuvzaQt9-XAg8e9Msw,--1UhMGODdWsrMastO9DZw,3.0
4,GL81ktDIteXA2VVH6gIakg,--1UhMGODdWsrMastO9DZw,5.0
...,...,...,...
99995,OsdBgzmbq4ZM93G3nk__gg,-pL4Id5cuz2yC2UKi2Yq9A,4.0
99996,1SmmRBCFST2fk930ifdbPw,-pL4Id5cuz2yC2UKi2Yq9A,4.0
99997,a6FN81i7fs4sUGuGfxFbmw,-pL4Id5cuz2yC2UKi2Yq9A,5.0
99998,-tAXa6cHTZizX42tcF0hpA,-pL4Id5cuz2yC2UKi2Yq9A,5.0


### 3. Reading in the dataset and prepping data

In [17]:
# Instantiate a 'Reader' to read in the data so Surprise can use it
reader = Reader(rating_scale=(1, 5))

# The columns must correspond to user id, item id and ratings (in that order).
data = Dataset.load_from_df(yelp[['user_id', 'business_id', 'stars']], reader)

In [18]:
trainset, testset = train_test_split(data, test_size=.2)

In [21]:
testset

[('2Z0AZvaaBq3uQLM9evYFYg', '-ITj6Pu8Gdw8MmLf0XBEKQ', 4.0),
 ('F-rUdtZCmvFYA6NiCBihvA', '-BxWyEIQ6wypT-37MzZizQ', 4.0),
 ('BGaKFrrIBMrOOa9Z4U3Pzw', '-4TMQnQJW1yd6NqGRDvAeA', 3.0),
 ('gyE_WebtGT29YoUbC2OuvA', '-9dmhyBvepc08KPEHlEM0w', 4.0),
 ('Da_ZO8lKwIacljqKUJbAlw', '-hsFN8VQvH26M0WZ3k1Wtg', 4.0),
 ('oFyOUOeGTRZhFPF9uTqrTQ', '-ZNTxbUVRg25LhOxEuW4rQ', 5.0),
 ('HVRueZq_6PsiipqT3pLVsw', '-4TMQnQJW1yd6NqGRDvAeA', 4.0),
 ('qf3X212Ojn4WJD2rflLTsA', '-fSW_9x8c_wTm1zS4dz3rw', 5.0),
 ('yT_QCcnq-QGipWWuzIpvtw', '-7EwIdxcRC5McO35DVfeSQ', 5.0),
 ('1YorWW0Z-YDuYC5GplNabw', '-ZBfr1BHvArFp1d6XH8jOQ', 4.0),
 ('fOYLR_xEYjlPgM8RFSUDZA', '-fOifZ2W5GhRHwXZEpBqCQ', 1.0),
 ('_wRDirL0O0eNQnq1497MPQ', '-a857YYdjzgOdOjFFRsRXQ', 2.0),
 ('FNX1z1h7GLY64J59CYo53A', '--FBCX-N37CMYDfs790Bnw', 1.0),
 ('4suJsf9Q2uruOwKlnBixow', '-bMZCfTK7fxFaURynKpBMA', 4.0),
 ('LOcJlhWSgjL6xW2EgdhI0g', '-jZ1TrvY56HFqmxjbpcU2A', 1.0),
 ('gXkpC3h5Svw5YjV2yHL0SQ', '-FRCGEz-NTsnhuITT5pYqg', 1.0),
 ('M1De2lTi1x9uzW1ah7FLEQ', '-PKUAor_Nz_

### 4. Fitting and evaluating models
Here, let's assume that we've tuned all these hyperparameters using GridSearch, and we've arrived at our final model.

In [22]:
final = SVD(n_epochs=20, n_factors=1, biased=True, 
              lr_all=0.005, reg_all=0.06)

In [23]:
final.fit(trainset)

In [24]:
predictions = final.test(testset)

In [25]:
predictions[:3]

[Prediction(uid='2Z0AZvaaBq3uQLM9evYFYg', iid='-ITj6Pu8Gdw8MmLf0XBEKQ', r_ui=4.0, est=3.8803452526760758, details={'was_impossible': False}),
 Prediction(uid='F-rUdtZCmvFYA6NiCBihvA', iid='-BxWyEIQ6wypT-37MzZizQ', r_ui=4.0, est=3.84820210467049, details={'was_impossible': False}),
 Prediction(uid='BGaKFrrIBMrOOa9Z4U3Pzw', iid='-4TMQnQJW1yd6NqGRDvAeA', r_ui=3.0, est=4.042315363310614, details={'was_impossible': False})]

In [26]:
accuracy.rmse(predictions)
accuracy.mae(predictions)

RMSE: 1.3009
MAE:  1.0556


1.0555675524631714

### 5. Making Predictions (again)
Unfortunately, this dataset has a convoluted string as the user/business IDs.

In [27]:
yelp['user_id'][55]

'HPtjvIrhzAUkKsiVkeT4MA'

In [28]:
yelp['business_id'][123]

'--7zmmkVg-IMGaXbuVd0SQ'

In [32]:
final.predict(yelp['user_id'][55], 3)

Prediction(uid='HPtjvIrhzAUkKsiVkeT4MA', iid=3, r_ui=None, est=3.537450123990713, details={'was_impossible': False})

### 6. What else?

Surprise has sample code where you can get the top **n** recommended items for a user. https://surprise.readthedocs.io/en/stable/FAQ.html

# Resources

- The structure of our lesson on recommendation engines is based on Chapter 9 of **Mining of Massive Datasets**: http://infolab.stanford.edu/~ullman/mmds/book.pdf
- Libraries for coding recommendation engines: 
    - Surprise: https://surprise.readthedocs.io/en/stable/index.html
    - LightFM: https://lyst.github.io/lightfm/docs/index.html
    
    
- Some blogs I might've written:
    - Overview: https://towardsdatascience.com/a-primer-to-recommendation-engines-49bd12ed849f?source=friends_link&sk=279dfeec5187614b37431dab167fd4e3
    - Collaborative filtering: https://towardsdatascience.com/a-primer-to-recommendation-engines-49bd12ed849f?source=friends_link&sk=279dfeec5187614b37431dab167fd4e3